In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.15,
                                         rotation_range=15,
                                         height_shift_range=0.15,
                                         width_shift_range=0.15,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [6]:
vgg19_base_model = VGG19(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg19_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

vgg19_model = Model(inputs=vgg19_base_model.input, outputs=x)

In [7]:
for layer in vgg19_model.layers[:22]:
    layer.trainable = False

for i,layer in enumerate(vgg19_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 flatten_1 True
23 dense_1 True
24 batch_normalization_1 True
25 dropout_1 True
26 dense_2 True
27 batch_normalization_2 True
28 dropout_2 True
29 dense_3 True


In [8]:
vgg19_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=5)])

vgg19_model.save('vgg19_model_fc.h5')

Epoch 1/15
156/156 [==============================] - 369s 2s/step - loss: 1.6290 - acc: 0.4868 - val_loss: 0.9036 - val_acc: 0.7188
Epoch 2/15
156/156 [==============================] - 365s 2s/step - loss: 0.5715 - acc: 0.8105 - val_loss: 0.7522 - val_acc: 0.8101
Epoch 3/15
156/156 [==============================] - 362s 2s/step - loss: 0.3832 - acc: 0.8753 - val_loss: 1.3856 - val_acc: 0.6777
Epoch 4/15
156/156 [==============================] - 358s 2s/step - loss: 0.3069 - acc: 0.9026 - val_loss: 0.9824 - val_acc: 0.7622
Epoch 5/15
156/156 [==============================] - 357s 2s/step - loss: 0.2577 - acc: 0.9173 - val_loss: 0.9428 - val_acc: 0.7565
Epoch 6/15
156/156 [==============================] - 358s 2s/step - loss: 0.2250 - acc: 0.9272 - val_loss: 1.3254 - val_acc: 0.7152
Epoch 7/15
156/156 [==============================] - 357s 2s/step - loss: 0.2103 - acc: 0.9325 - val_loss: 1.3454 - val_acc: 0.7108


In [14]:
# vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #,decay=1e-6,nesterov=True),
#                     #optimizer=optimizers.Adam(lr=1e-3),
# #                     optimizer=Adadelta(),
#                     metrics=['accuracy'])

# # fine-tune the model
# vgg19_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=10,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[EarlyStopping(patience=5)])

In [10]:
# vgg19_model.save('vgg19_model_fc.h5')

In [10]:
vgg19_model = load_model('vgg19_model_fc.h5')

for layer in vgg19_model.layers[:17]:
    layer.trainable = False
    
for layer in vgg19_model.layers[17:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=5)])

vgg19_model.save('vgg19_model_17.h5')

Epoch 1/20
156/156 [==============================] - 361s 2s/step - loss: 0.1704 - acc: 0.9455 - val_loss: 0.8922 - val_acc: 0.7843
Epoch 2/20
156/156 [==============================] - 358s 2s/step - loss: 0.0709 - acc: 0.9786 - val_loss: 0.8188 - val_acc: 0.8436
Epoch 3/20
156/156 [==============================] - 356s 2s/step - loss: 0.0520 - acc: 0.9850 - val_loss: 0.8832 - val_acc: 0.7827
Epoch 4/20
156/156 [==============================] - 352s 2s/step - loss: 0.0300 - acc: 0.9914 - val_loss: 0.8014 - val_acc: 0.8367
Epoch 5/20
156/156 [==============================] - 357s 2s/step - loss: 0.0442 - acc: 0.9875 - val_loss: 1.2659 - val_acc: 0.8445
Epoch 6/20
156/156 [==============================] - 364s 2s/step - loss: 0.0302 - acc: 0.9909 - val_loss: 0.8130 - val_acc: 0.8271
Epoch 7/20
156/156 [==============================] - 360s 2s/step - loss: 0.0210 - acc: 0.9937 - val_loss: 0.5826 - val_acc: 0.8724
Epoch 8/20
156/156 [==============================] - 361s 2s/step - 

In [11]:
vgg19_model = load_model('vgg19_model_17.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=5)])

vgg19_model.save('vgg19_model_12.h5')

Epoch 1/20
156/156 [==============================] - 377s 2s/step - loss: 0.0690 - acc: 0.9780 - val_loss: 0.5876 - val_acc: 0.8863
Epoch 2/20
156/156 [==============================] - 371s 2s/step - loss: 0.0348 - acc: 0.9896 - val_loss: 0.4925 - val_acc: 0.8576
Epoch 3/20
156/156 [==============================] - 372s 2s/step - loss: 0.0428 - acc: 0.9872 - val_loss: 0.9329 - val_acc: 0.8545
Epoch 4/20
156/156 [==============================] - 372s 2s/step - loss: 0.0234 - acc: 0.9927 - val_loss: 1.1357 - val_acc: 0.8380
Epoch 5/20
156/156 [==============================] - 372s 2s/step - loss: 0.0268 - acc: 0.9917 - val_loss: 0.7727 - val_acc: 0.8432
Epoch 6/20
156/156 [==============================] - 371s 2s/step - loss: 0.0231 - acc: 0.9935 - val_loss: 0.4792 - val_acc: 0.8855
Epoch 7/20
156/156 [==============================] - 373s 2s/step - loss: 0.0199 - acc: 0.9949 - val_loss: 0.9648 - val_acc: 0.8262
Epoch 8/20
156/156 [==============================] - 371s 2s/step - 

In [12]:
# vgg19_model.compile(loss='categorical_crossentropy',
# #                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(lr=1e-4),
#                     metrics=['accuracy'])  

# vgg19_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=50,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
#                                    CSVLogger('vgg19_fine_tune'), #])
#                                    EarlyStopping(patience=10)])

In [13]:
# vgg19_model.save('vgg19_model_final.h5')

In [15]:
vgg19_model = load_model('vgg19_model_12.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),#decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune')])
#                                    EarlyStopping(patience=3)])

vgg19_model.save('vgg19_final_model.h5')

Epoch 1/50
156/156 [==============================] - 372s 2s/step - loss: 0.0090 - acc: 0.9972 - val_loss: 0.4806 - val_acc: 0.9080
Epoch 2/50
156/156 [==============================] - 373s 2s/step - loss: 0.0099 - acc: 0.9966 - val_loss: 0.5491 - val_acc: 0.8937
Epoch 3/50
156/156 [==============================] - 365s 2s/step - loss: 0.0070 - acc: 0.9983 - val_loss: 0.6186 - val_acc: 0.8759
Epoch 4/50
156/156 [==============================] - 365s 2s/step - loss: 0.0081 - acc: 0.9975 - val_loss: 0.6585 - val_acc: 0.8685
Epoch 5/50
156/156 [==============================] - 366s 2s/step - loss: 0.0085 - acc: 0.9973 - val_loss: 0.6364 - val_acc: 0.8733
Epoch 6/50
156/156 [==============================] - 367s 2s/step - loss: 0.0081 - acc: 0.9979 - val_loss: 0.5697 - val_acc: 0.8868
Epoch 7/50
156/156 [==============================] - 372s 2s/step - loss: 0.0079 - acc: 0.9976 - val_loss: 0.6478 - val_acc: 0.8728
Epoch 8/50
156/156 [==============================] - 371s 2s/step - 

In [16]:
vgg19_model.save('vgg19_final_model.h5')

In [17]:
vgg19_model = load_model('vgg19_final_model.h5')

test_model(vgg19_model,test_path,csv='vgg19_submission.csv')